In [1]:
import pandas as pd
import cufflinks as cf
import mytools.date as dt
import mytools.plot as mpl

cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

## Italian intensive care cases 

In [2]:
# loading the data
df = pd.read_csv('italy-intensive_care.csv')
a = list((dt.day_of_year_to_date(v)).strftime("%d/%m/%y") for v in df['day'])
df.index = a
df.drop(columns=['day'])
# display the table


,intensive care
24/02/20,27
25/02/20,35
26/02/20,36
27/02/20,56
28/02/20,64
29/02/20,105
01/03/20,140
02/03/20,166
03/03/20,229
04/03/20,295


## Fitting intensive care data with exponential and logistic models

The exponential model is:

$$y = e^{k(x-x_0)} + y_0$$

where $k, x_0, y_0$ are the parameters to estimate.

For the logistic model:

$$y = \frac{c}{1+e^{-k(x-x_0)}} + y_0$$

where $c, k, x_0, y_0$ are the parameters to estimate.

In [3]:
fig = mpl.iplot_analysis_plot(df.drop(columns=['day']), title='Intensive care cases')
fig.show()

[0.65       0.47186597 1.        ]
`ftol` termination condition is satisfied.
Function evaluations 17, initial cost 1.1693e+01, final cost 6.6719e-03, first-order optimality 3.35e-08.
            Exponential model
            x0 = -54.099013760891836
            y0 = -908.5680655486492
            k = 0.0607109097255004
            
`gtol` termination condition is satisfied.
Function evaluations 21, initial cost 3.0932e+00, final cost 2.0516e-04, first-order optimality 7.32e-10.
        Sigmoid model
        x0 = 77.80647478198152
        y0 = -95.10176577963692
        c = 4651.4994699145445
        k = 0.17144504031138652
        asymptot = 4556.397704134908
        flex = 77.80647478198152, 2230.6479691776353
        
`ftol` termination condition is satisfied.
Function evaluations 24, initial cost 1.2528e+00, final cost 3.4613e-04, first-order optimality 1.56e-08.
        Sigmoid derivative model
        x0 = 87.64165209487376
        y0 = -162.89952545054527
        c = 105496.9585